In [5]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i, end=' ')
    def writeln(): print()
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

compileString("""
type A = [1.. 7] → integer
var x: A
var y: A
var i: integer
var h: integer
program p
    i := 3
    h := y[i]
    x[i] := 5
""", "partialArray.wat", target = 'wat')

#compileString("""
#type A = [1↦4,3↦7] → integer
#var x: A
#var y: A
#var i: integer
#var h: integer
#program p
#  var i: integer
#  var z: A
#  write(z[1])
#  x[2] := z[1]
#  write(x[2])
#""", "partialArray.wat", target = 'wat')

#compileString("""
#type A = [1↦4,3↦7] → integer
#var x: A
#var y: A
#var i: integer
#var h: integer
#program p
#  var i: integer
#  var z: A
#  for a in x do
#    write(a)
#""", "partialArray.wat", target = 'wat')

In [6]:
!wat2wasm partialArray.wat

In [7]:
cat partialArray.wat

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $i (mut i32) i32.const 0)
(global $h (mut i32) i32.const 0)
(global $_memsize (mut i32) i32.const 56)
(func $program
(local $0 i32)
i32.const 3
global.set $i
global.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 28
i32.add
i32.load
global.set $h
global.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add
i32.const 5
i32.store
)
(memory 1)
(start $program)
)

In [8]:
runwasmer("partialArray.wasm")